In [8]:
import datetime
import os

from ml.ml_core.src.core.model_factory import model_factory

os.getcwd()
if '.git' not in os.listdir():
    os.chdir('./../../../../')
os.listdir()

import fasttext
import keras_core as keras
import numpy as np
import pandas as pd
from keras_core import Sequential
from keras_core.src.layers import Dense
from keras_core.models import load_model
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorboard.plugins.hparams import api as hp

## import fastTest model

## Configurations

In [5]:
params_grid = {
    "model": "mlp",
    "embedder" : "fasttext",
    "embedding_data" : "cc.fr.300.bin",
    "dataset" : "fr_hf.csv",
    "batch_size" : 300
}

In [6]:
data_folder_src = 'ml/ml_core/data/processed/'
ml_core_folder_path = 'ml/ml_core'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"{ml_core_folder_path}log/fit/{experiment_name}"
model_weight_dst = f'{ml_core_folder_path}model_weights/{experiment_name}-model.keras'

In [7]:

vector_path = f'{ml_core_folder_path}embedded_vector/{params_grid["embedding_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'

# loading dataset

In [6]:
X = np.load(vector_x_dst)
y = np.load(vector_y_dst)

## building and training the model

### callbacks

In [8]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [9]:

class EarlyStoppingLogging(tf.keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [10]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

2024-07-02 13:57:41.539191: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 13:57:41.778412: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 13:57:41.778453: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 13:57:41.805284: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 13:57:41.805358: I tensorflow/compile

In [11]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

### training the model

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = model_factory(params_grid['model'], X_train.shape[1], 1)

model.fit(X_train,
          y_train,
          epochs=500,
          batch_size = params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

/home/arys/miniconda3/envs/DEV_IA/lib/python3.10/site-packages/keras_core/src/layers/core/dense.py:73: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500


2024-07-02 13:57:44.890574: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x12ac4340 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-02 13:57:44.890619: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-02 13:57:44.933816: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-02 13:57:45.150477: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


36/49 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6861 - loss: 0.6324

2024-07-02 13:57:45.792936: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/49 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.7025 - loss: 0.6181 - val_accuracy: 0.7687 - val_loss: 0.5315
Epoch 2/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7616 - loss: 0.5336 - val_accuracy: 0.7687 - val_loss: 0.5042
Epoch 3/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7635 - loss: 0.5023 - val_accuracy: 0.7713 - val_loss: 0.4806
Epoch 4/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7637 - loss: 0.4907 - val_accuracy: 0.7815 - val_loss: 0.4690
Epoch 5/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7877 - loss: 0.4626 - val_accuracy: 0.7797 - val_loss: 0.4670
Epoch 6/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7894 - loss: 0.4568 - val_accuracy: 0.7926 - val_loss: 0.4606
Epoch 7/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7914 - loss: 0.4462 - val_accuracy: 0.7690 - val_loss: 0.4880
Epoch 8/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7910 - loss: 0.4426 - val_accuracy: 0.7708 - val_loss: 0.

# save model

In [ ]:
model.save(model_weight_dst)